In [4]:
import pandas as pd
import numpy as np
import random
#from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
HotelSearch_orig=pd.read_csv("/Users/sureshvadakath/Documents/Datasets/SearchRanking/HotelSearch_train.csv")

In [7]:
HotelSearch_orig.dtypes
def get_target(row):
    """
    0=not clicked at all, 1=clicked but not booked, 2=booked
    """
#     if row.booking_bool>0:
#         return 2
#     if row.click_bool>0 :
#         return 1
#     return 0

    if row.booking_bool>0:
        return 1
    return 0

#print(HotelSearch_orig['booking_bool'].unique())
HotelSearch_orig["date_time"] = pd.to_datetime(HotelSearch_orig["date_time"],infer_datetime_format=True)
HotelSearch_orig["target"] = HotelSearch_orig.apply(get_target,axis=1)

HotelSearch_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917530 entries, 0 to 9917529
Data columns (total 55 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   srch_id                      int64         
 1   date_time                    datetime64[ns]
 2   site_id                      int64         
 3   visitor_location_country_id  int64         
 4   visitor_hist_starrating      float64       
 5   visitor_hist_adr_usd         float64       
 6   prop_country_id              int64         
 7   prop_id                      int64         
 8   prop_starrating              int64         
 9   prop_review_score            float64       
 10  prop_brand_bool              int64         
 11  prop_location_score1         float64       
 12  prop_location_score2         float64       
 13  prop_log_historical_price    float64       
 14  position                     int64         
 15  price_usd                    float64       
 16  

In [8]:
HotelSearch_all=HotelSearch_orig.dropna(axis=1,how="all")
HotelSearch_all.shape


(9917530, 55)

In [9]:
All_Searches=HotelSearch_all["srch_id"].unique().tolist()
training_sample_list = random.sample(All_Searches, int(len(All_Searches)*.01))
print(len(All_Searches))
print(len(training_sample_list))
test_sample_list_temp=set(All_Searches) - set(training_sample_list)
test_sample_list = random.sample(test_sample_list_temp, int(len(test_sample_list_temp)*.015))
print(len(test_sample_list_temp))
print(len(test_sample_list))
#training_searches_array = np.array(training_sample_list)
#test_searches_array = np.array(test_sample_list)


399344
3993
395351
5930


/var/folders/kc/v7s5qpm17l14gx5297w1_6pw0000gn/T/ipykernel_87181/1782490068.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_sample_list = random.sample(test_sample_list_temp, int(len(test_sample_list_temp)*.015))


In [23]:
#HotelSearch_train = HotelSearch_all.groupby('srch_id').filter(lambda x: x["srch_id"].values[0] in sampled_searches)
HotelSearch_train_temp = HotelSearch_all[HotelSearch_all['srch_id'].isin(training_sample_list)]
# Not in Test:  position, click_bool, gross_bookings_usd, nor booking_bool
HotelSearch_test_temp = HotelSearch_all[HotelSearch_all['srch_id'].isin(test_sample_list)]
print(HotelSearch_train_temp.shape)
print(HotelSearch_test_temp.shape)



(98361, 55)
(147730, 55)


In [30]:
HotelSearch_train_temp=HotelSearch_train_temp.sort_values(by=['srch_id','position'], ascending=True)
HotelSearch_train_temp = HotelSearch_train_temp.reset_index(drop=True)
HotelSearch_train_temp.shape
HotelSearch_train_temp['predictionid'] = np.arange(len(HotelSearch_train_temp))
HotelSearch_train_temp.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,target,predictionid
0,71,2013-03-12 23:14:50,24,216,NaN,NaN,98,76541,3,3.5,0,2.20,0.0482,3.21,1,16.90,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0
1,71,2013-03-12 23:14:50,24,216,NaN,NaN,98,37933,3,4.0,0,2.08,0.0812,3.59,2,18.34,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,46.0,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,1
2,71,2013-03-12 23:14:50,24,216,NaN,NaN,98,108588,4,4.0,0,0.00,0.0268,4.08,3,26.78,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,108.0,1.0,0.0,32.0,-1.0,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,2
3,71,2013-03-12 23:14:50,24,216,NaN,NaN,98,28672,3,4.0,0,2.20,0.0151,3.35,4,21.48,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,3
4,71,2013-03-12 23:14:50,24,216,NaN,NaN,98,25741,3,3.5,0,1.61,0.0359,4.02,6,19.47,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,10.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,4


In [31]:
HotelSearch_test_temp=HotelSearch_test_temp.sort_values(by=['srch_id','position'], ascending=True)
HotelSearch_test_temp = HotelSearch_test_temp.reset_index(drop=True)
HotelSearch_test_temp['predictionid'] = np.arange(len(HotelSearch_test_temp))
HotelSearch_test_temp.shape

(147730, 56)

In [34]:
HotelSearch_train_temp.to_csv('./data/HotelSearch_train_sampled.csv')
HotelSearch_test_temp.to_csv('./data/HotelSearch_test_sampled.csv')


In [33]:
HotelSearch_test_temp.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,target,predictionid
0,246,2013-03-04 09:55:43,14,100,NaN,NaN,219,11396,4,5.0,0,0.69,0.0551,5.36,1,159.64,0,18294,1,6,2,0,1,0,NaN,87.17,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,181.32,1,1,0
1,246,2013-03-04 09:55:43,14,100,NaN,NaN,219,103885,2,4.0,1,1.10,0.1540,4.75,2,98.32,0,18294,1,6,2,0,1,0,NaN,93.87,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,1
2,246,2013-03-04 09:55:43,14,100,NaN,NaN,219,61167,3,4.5,0,1.10,0.0314,4.87,3,101.24,0,18294,1,6,2,0,1,0,NaN,92.31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,2
3,246,2013-03-04 09:55:43,14,100,NaN,NaN,219,95490,2,3.5,1,1.10,0.0142,4.88,4,95.40,0,18294,1,6,2,0,1,0,NaN,89.28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,3
4,246,2013-03-04 09:55:43,14,100,NaN,NaN,219,127213,3,4.0,1,1.10,0.1652,4.95,6,119.73,0,18294,1,6,2,0,1,0,NaN,93.97,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,4
